# Run inference on LDM

### Installation

Only for colab

In [ ]:
!git clone --recurse-submodules https://github.com/rossiyareich/marching-waifu.git
%cd marching-waifu

In [ ]:
# Download pretrained DeepDanbooru weights
!mkdir ext/weights/
%cd ext/weights/
!wget https://github.com/KichangKim/DeepDanbooru/releases/download/v3-20211112-sgd-e28/deepdanbooru-v3-20211112-sgd-e28.zip
!unzip deepdanbooru-v3-20211112-sgd-e28.zip -d ../AnimeFaceNotebooks/deepdanbooru_model/
%cd ../
!rm -rf weights
%cd ../

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Only for local install (run `setup.sh` or `setup.bat` and `huggingface-cli login` beforehand!)

In [ ]:
%cd ../

Project setup

In [ ]:
!nvidia-smi

In [ ]:
# Install requirements
!python -m pip install -r requirements.txt
!python -m pip install -r ext/Real-ESRGAN/requirements.txt

# Create directories
!mkdir data/ngp/overview/
!mkdir data/ngp/train/

# Install local packages
%cd ext/Real-ESRGAN/
!python setup.py develop
%cd ../AnimeFaceNotebooks/DeepDanbooru/
!python setup.py develop
%cd ../../../

In [ ]:
%cd scripts

In [ ]:
import sys

sys.path.append("..")

### Configuration

In [ ]:
import json

# @title Inference configuration
class config(object):
    pass

# @markdown ###**Models**
models = config()
models.ldm_repo_id = "rossiyareich/Nabylon-v1.0-fp16"  # @param {type: "string"}
models.ldm_inpaint_repo_id = "rossiyareich/Nabylon-v1.0-fp16-inpainting"  # @param {type: "string"}
models.vae_repo_id = "stabilityai/sd-vae-ft-mse"  # @param {type: "string"}

# @markdown ###**ControlNet**
controlnet = config()
controlnet.unit_scales = config()
controlnet.guidance = config()
controlnet.soft_exp = 0.825  # @param {type:"slider", min:0, max:1, step:0.025}
controlnet.unit_scales.openpose = 1.0  # @param {type:"slider", min:0, max:2, step:0.1}
controlnet.unit_scales.depth = 0.8  # @param {type:"slider", min:0, max:2, step:0.1}
controlnet.unit_scales.normals = 0.8  # @param {type:"slider", min:0, max:2, step:0.1}
controlnet.unit_scales.lineart = 0.6  # @param {type:"slider", min:0, max:2, step:0.1}
controlnet.guidance.start = 0.0 # @param {type:"slider", min:0, max:1, step:0.1}
controlnet.guidance.end = 1.0 # @param {type:"slider", min:0, max:1, step:0.1}

# @markdown ###**DeepDanbooru**
deepdanbooru = config()
deepdanbooru.threshold = 0.08  # @param {type:"slider", min:0, max:1, step:0.01}
deepdanbooru.multiplier = 1.0  # @param {type:"slider", min:0, max:4, step:0.1}
deepdanbooru.prefix = "(masterpiece, best quality, {0})+, "  # @param {type: "string"}

# @markdown ###**Pipeline**
pipeline = config()
# @markdown #####**PrereqGen**
pipeline.prereq = config()
pipeline.prereq.prompt = "(masterpiece, best quality, dark background, {0})+, 1girl, white hoodie, earmuffs, leggings, white scarf, black gloves, white socks, short blue hair, blue eyes, bangs"  # @param {type: "string"}
pipeline.prereq.negative_prompt = "EasyNegative, (worst quality, low quality, logo, text, watermark, username, nsfw), inaccurate hands and fingers"  # @param {type: "string"}
pipeline.prereq.steps = 20  # @param {type:"slider", min:1, max:150, step:1}
pipeline.prereq.cfg_scale = 10  # @param {type:"slider", min:1, max:30, step:0.5}
pipeline.prereq.callback_steps = 5  # @param {type:"slider", min:0, max:150, step:1}
pipeline.prereq.denoising_strength = 1.0  # @param {type:"slider", min:0, max:1, step:0.5}
pipeline.prereq.seed = -1  # @param {type:"integer"}
# @markdown #####**RestGen**
pipeline.restgen = config()
pipeline.restgen.negative_prompt = "EasyNegative, (worst quality, low quality, logo, text, watermark, username, nsfw), inaccurate hands and fingers"  # @param {type: "string"}
pipeline.restgen.steps = 20  # @param {type:"slider", min:1, max:150, step:1}
pipeline.restgen.cfg_scale = 10  # @param {type:"slider", min:1, max:30, step:0.5}
pipeline.restgen.callback_steps = 0  # @param {type:"slider", min:0, max:150, step:1}
pipeline.restgen.denoising_strength = 1.0  # @param {type:"slider", min:0, max:1, step:0.5}
pipeline.restgen.seed = -1  # @param {type:"integer"}
pipeline.restgen.inpaint_method = "original"  # @param {type:"string"}
pipeline.restgen.dataset_size = 40  # @param {type:"integer"}

inference = {
    "models": {
        "ldm_repo_id": models.ldm_repo_id,
        "ldm_inpaint_repo_id": models.ldm_inpaint_repo_id,
        "vae_repo_id": models.vae_repo_id,
    },
    "controlnet": {
        "soft_exp": controlnet.soft_exp,
        "unit_scales": {
            "openpose": controlnet.unit_scales.openpose,
            "depth": controlnet.unit_scales.depth,
            "normals": controlnet.unit_scales.normals,
            "lineart": controlnet.unit_scales.lineart,
        },
        "guidance": {
            "start": controlnet.guidance.start,
            "end": controlnet.guidance.end,
        },
    },
    "deepdanbooru": {
        "threshold": deepdanbooru.threshold,
        "multiplier": deepdanbooru.multiplier,
        "prefix": deepdanbooru.prefix,
    },
    "pipeline": {
        "prereq": {
            "prompt": pipeline.prereq.prompt,
            "negative_prompt": pipeline.prereq.negative_prompt,
            "steps": pipeline.prereq.steps,
            "cfg_scale": pipeline.prereq.cfg_scale,
            "callback_steps": pipeline.prereq.callback_steps,
            "denoising_strength": pipeline.prereq.denoising_strength,
            "seed": pipeline.prereq.seed,
        },
        "restgen": {
            "negative_prompt": pipeline.restgen.negative_prompt,
            "steps": pipeline.restgen.steps,
            "cfg_scale": pipeline.restgen.cfg_scale,
            "callback_steps": pipeline.restgen.callback_steps,
            "denoising_strength": pipeline.restgen.denoising_strength,
            "seed": pipeline.restgen.seed,
            "inpaint_method": pipeline.restgen.inpaint_method,
            "dataset_size": pipeline.restgen.dataset_size,
        },
    },
}

with open("inference.json", "w") as f:
    json.dump(inference, f, indent=4)

### Inference

In [ ]:
import os

import PIL.Image

from src.utils.file_loader import *
from src.utils.image_wrapper import *

path = {
    "controlnet_conditions_folder": "../data/multi_controlnet/multi_controlnet_data/",
    "ngp_overview_folder": "../data/ngp/overview/",
    "ngp_train_folder": "../data/ngp/train/",
}

def display_conditions(conditions, scale):
    img = image_wrapper(conditions[0], format="pil").scale(scale)
    for img_ in conditions[1:]:
        img.concatenate(image_wrapper(img_, format="pil").scale(scale))
    display(img.to_pil())

def display_at_index(index, filename, scale, show_overview):
    img = PIL.Image.open(os.path.join(path["ngp_train_folder"], filename))
    display(
        image_wrapper(img, format="pil")
        .scale(scale)
        .to_pil()
    )

    display_conditions(controlnet_conditions[index], scale)
    
    if show_overview:
        interim = PIL.Image.open(os.path.join(path["ngp_overview_folder"], filename))
        display(
            image_wrapper(interim, format="pil")
            .scale(img.width * scale / interim.width)
            .to_pil()
        )

fl = file_loader()
controlnet_conditions = fl.load_controlnet_conditions(path["controlnet_conditions_folder"])

##### Run Prereq

In [ ]:
!python generation_prereq.py

In [ ]:
scale = 0.25
display_at_index(0, "prereq.png", scale, pipeline.prereq.callback_steps > 0)

##### Run RestGen

In [ ]:
!python generation_restgen.py

In [ ]:
import time

from IPython.display import clear_output

scale = 0.25
display_interval = 1

for i in range(0, pipeline.restgen.dataset_size):
    display_at_index(i, f"{(i+1):04}.png", scale, pipeline.restgen.callback_steps > 0)
    time.sleep(display_interval)
    if i < (pipeline.restgen.dataset_size - 1):
        clear_output(True)

# Run training on NeRF

# Extract mesh from trained NeRF

# Run inference on T2M-GPT

# Retarget animation on to character mesh

# Evaluation